# 05 - Backtest Analysis

Deep-dive into backtest results with regime-conditional analysis.

**Contents:**
- Load backtest results and metrics
- Equity curve analysis
- Drawdown analysis
- Monthly returns heatmap
- Regime-conditional performance breakdown
- RL agent vs baseline comparison

In [ ]:
# === Colab Auto-Detection ===
import sys, os
if "google.colab" in sys.modules:
    import subprocess
    if not os.path.exists("/content/quant-lab"):
        subprocess.run(["git", "clone", "https://github.com/Mohit1053/quant-lab.git", "/content/quant-lab"], check=True)
    os.chdir("/content/quant-lab")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-e", "."], check=True)
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    # Symlink data from Drive if available
    from pathlib import Path
    drive_data = Path("/content/drive/MyDrive/quant_lab/data")
    if drive_data.exists():
        import shutil
        for sub in ["raw", "cleaned", "features"]:
            src = drive_data / sub
            dst = Path("data") / sub
            if src.exists():
                dst.mkdir(parents=True, exist_ok=True)
                for f in src.glob("*.parquet"):
                    shutil.copy(f, dst / f.name)
    print("Colab setup complete!")
else:
    sys.path.insert(0, "../src")


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from quant_lab.backtest.metrics import compute_all_metrics

In [ ]:
# Load backtest results (or generate demo data)
equity_path = '../outputs/backtests/equity_curve.parquet'
try:
    equity_df = pd.read_parquet(equity_path)
    portfolio_values = equity_df['equity'].values
    dates = pd.to_datetime(equity_df.index)
except FileNotFoundError:
    print('No saved backtest. Using simulated data for demo.')
    np.random.seed(42)
    dates = pd.bdate_range('2023-07-01', periods=252)
    returns = np.random.randn(252) * 0.01 + 0.0002
    portfolio_values = 1_000_000 * np.cumprod(1 + returns)

returns = np.diff(portfolio_values) / portfolio_values[:-1]
print(f'Period: {dates[0].date()} to {dates[-1].date()}')
print(f'Final value: {portfolio_values[-1]:,.0f}')

In [ ]:
# Equity curve
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=portfolio_values, mode='lines', name='Portfolio'))
fig.update_layout(title='Equity Curve', height=400, template='plotly_white',
                  xaxis_title='Date', yaxis_title='Portfolio Value')
fig.show()

In [ ]:
# Drawdown
peak = np.maximum.accumulate(portfolio_values)
drawdown = (portfolio_values - peak) / peak

fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=drawdown, fill='tozeroy', name='Drawdown',
                          line=dict(color='red', width=1)))
fig.update_layout(title='Drawdown', height=300, template='plotly_white',
                  yaxis_tickformat='.1%')
fig.show()

print(f'Max drawdown: {drawdown.min():.2%}')

In [ ]:
# Monthly returns heatmap
ret_series = pd.Series(returns, index=dates[1:])
monthly = ret_series.resample('ME').sum()
monthly_df = pd.DataFrame({
    'year': monthly.index.year,
    'month': monthly.index.month,
    'return': monthly.values,
})
heatmap_data = monthly_df.pivot(index='year', columns='month', values='return')

fig = px.imshow(heatmap_data, title='Monthly Returns Heatmap',
                color_continuous_scale='RdYlGn', zmin=-0.1, zmax=0.1,
                labels={'x': 'Month', 'y': 'Year', 'color': 'Return'})
fig.update_layout(height=300)
fig.show()

In [ ]:
# Performance metrics
equity_series = pd.Series(portfolio_values, index=dates)
returns_series = pd.Series(returns, index=dates[1:])
metrics = compute_all_metrics(equity_series, returns_series)

print('\nPerformance Metrics:')
print('=' * 40)
for k, v in metrics.items():
    if 'return' in k or 'cagr' in k or 'drawdown' in k:
        print(f'{k:25s}: {v:>10.2%}')
    else:
        print(f'{k:25s}: {v:>10.4f}')

In [ ]:
# Regime-conditional analysis (if available)
try:
    regime_df = pd.read_parquet('../outputs/regimes/regime_labels.parquet')
    labels = regime_df['regime_label'].values[:len(returns)]

    for rid in sorted(set(labels[labels >= 0])):
        mask = labels == rid
        r = returns[mask]
        print(f'\nRegime {rid}: {mask.sum()} days ({mask.sum()/len(labels)*100:.1f}%)')
        print(f'  Ann. Return: {np.mean(r) * 252:.2%}')
        print(f'  Ann. Vol:    {np.std(r) * np.sqrt(252):.2%}')
        sharpe = np.mean(r) * 252 / (np.std(r) * np.sqrt(252)) if np.std(r) > 0 else 0
        print(f'  Sharpe:      {sharpe:.2f}')
except FileNotFoundError:
    print('No regime labels found. Run: python scripts/detect_regimes.py')